In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import seaborn as sns
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from scipy.stats import f_oneway
from collections import defaultdict
from load_data import load_data,get_SG_FR, similarity_metric, read_Decoding_csv, load_neural_npz
from dataset import LinearRegressionModel, NeuralDataset

e:\miniconda\conda\envs\SCNS\lib\site-packages\oasis\functions.py:13: UserWarning: Could not find cvxpy. Don't worry, you can still use OASIS, just not the slower interior point methods we compared to in the papers.
  warn("Could not find cvxpy. Don't worry, you can still use OASIS, " +


In [2]:
# load data
Target_Brain_region = 'VISp'
data_root = '../data'
sessions = ['A','B']
session_datas = []
session_folders = [] 
for session in sessions:
    tgt_dict, tgt_folder = read_Decoding_csv(data_path='../data',region_type = Target_Brain_region, session_letter = session)
    session_datas.append(tgt_dict)
    session_folders.append(tgt_folder)
print('================================================')
print(session_folders)
ts_sessions = []
dff_sessions = []
all_roi_masks_sessions = []
cids_sessions = []
metadata_sessions = []

for tgt_data_path in session_folders:
    for file_name in os.listdir(os.path.join(data_root, tgt_data_path)):
        if file_name.endswith('.npz'):
            ts, dff, all_roi_masks, cids, metadata = load_neural_npz(os.path.join(data_root, tgt_data_path,file_name))
            ts_sessions.append(ts)
            dff_sessions.append(dff)
            all_roi_masks_sessions.append(all_roi_masks)
            cids_sessions.append(cids)
            metadata_sessions.append(metadata)
            
            position = metadata.tolist()
            print('================================================')
            print('Caution, the position of this file is about the brain part of', position['targeted_structure'])
            print('================================================')
print('================================================')

we have keys in data_dict, including:
stimulus_metadata_drifting_gratings.csv
stimulus_metadata_natural_movie_one.csv
stimulus_metadata_natural_movie_three.csv
stimulus_metadata_spontaneous.csv
we have keys in data_dict, including:
stimulus_metadata_natural_movie_one.csv
stimulus_metadata_natural_scenes.csv
stimulus_metadata_spontaneous.csv
stimulus_metadata_static_gratings.csv
stimulus_metadata_total.csv
['VISp_three_session_A_501704220', 'VISp_three_session_B_501559087']
check subtitles, should be ['ts', 'dff', 'all_roi_masks', 'cids', 'metadata']
['ts', 'dff', 'all_roi_masks', 'cids', 'metadata']
Caution, the position of this file is about the brain part of VISp
check subtitles, should be ['ts', 'dff', 'all_roi_masks', 'cids', 'metadata']
['ts', 'dff', 'all_roi_masks', 'cids', 'metadata']
Caution, the position of this file is about the brain part of VISp


In [3]:
print()